# 📊 Scraping - Fusión de perfiles seguidores

Notebook estilo Colab para agregar los **16 perfiles faltantes** a `perfilesSeguidores`.

---

## En Google Colab
1. Sube la carpeta del proyecto o monta tu Drive
2. Ajusta la ruta `BASE_PATH` en la celda de configuración
3. Ejecuta todas las celdas

## 1. Configuración y rutas

In [2]:
# Ruta base del proyecto
# En local: usa la ruta de tu carpeta scraping
# En Colab: '/content/drive/MyDrive/scraping' o '/content/scraping' según donde subas los archivos

import os
import json
import pandas as pd

# Para Colab con Google Drive descomenta y ejecuta:
# from google.colab import drive
# drive.mount('/content/drive')

BASE_PATH = '/Users/geotrends/Desktop/scraping'  # Cambiar en Colab
SCRAPERS_PATH = os.path.join(BASE_PATH, 'scrapers')

FALTANTES_FILE = os.path.join(BASE_PATH, '16faltantes.json')
PERFILES_JSON = os.path.join(SCRAPERS_PATH, 'perfilesSeguidores.json')
PERFILES_CSV = os.path.join(SCRAPERS_PATH, 'perfilesSeguidores.csv')

print(f'📁 Base path: {BASE_PATH}')
print(f'✅ 16faltantes existe: {os.path.exists(FALTANTES_FILE)}')
print(f'✅ perfilesSeguidores.json existe: {os.path.exists(PERFILES_JSON)}')

📁 Base path: /Users/geotrends/Desktop/scraping
✅ 16faltantes existe: True
✅ perfilesSeguidores.json existe: True


## 2. Cargar datos

In [3]:
# Cargar 16 faltantes
with open(FALTANTES_FILE, 'r', encoding='utf-8') as f:
    faltantes = json.load(f)

# Cargar perfiles seguidores actuales
with open(PERFILES_JSON, 'r', encoding='utf-8') as f:
    perfiles = json.load(f)

print(f'📋 Perfiles faltantes a incorporar: {len(faltantes)}')
print(f'📋 Perfiles actuales en perfilesSeguidores: {len(perfiles)}')

📋 Perfiles faltantes a incorporar: 16
📋 Perfiles actuales en perfilesSeguidores: 8855


## 3. Estructura estándar y normalización

Algunos perfiles en `16faltantes` pueden tener campos faltantes (ej: solo `username`). Los normalizamos para que coincidan con la estructura de `perfilesSeguidores`.

In [4]:
# Estructura esperada de cada perfil (como en perfilesSeguidores)
CAMPOS_ESPERADOS = [
    'fullName', 'profilePicUrl', 'username', 'postsCount', 'followersCount',
    'followsCount', 'private', 'verified', 'isBusinessAccount', 'biography'
]

def normalizar_perfil(p):
    """Asegura que el perfil tenga todos los campos con valores por defecto."""
    defaults = {
        'fullName': '',
        'profilePicUrl': '',
        'username': p.get('username', ''),
        'postsCount': 0,
        'followersCount': 0,
        'followsCount': 0,
        'private': False,
        'verified': False,
        'isBusinessAccount': False,
        'biography': ''
    }
    return {k: p.get(k, defaults[k]) for k in CAMPOS_ESPERADOS}

# Normalizar los 16 faltantes
faltantes_norm = [normalizar_perfil(p) for p in faltantes]
print('Perfiles normalizados. Ejemplo del primero:')
print(json.dumps(faltantes_norm[0], indent=2, ensure_ascii=False)[:500], '...')

Perfiles normalizados. Ejemplo del primero:
{
  "fullName": "Carlos Andres Restrepo Mora",
  "profilePicUrl": "https://scontent-msp1-1.cdninstagram.com/v/t51.2885-19/639763142_18567115672024206_6320678634278112133_n.jpg?stp=dst-jpg_e0_s150x150_tt6&efg=eyJ2ZW5jb2RlX3RhZyI6InByb2ZpbGVfcGljLmRqYW5nby4xMDgwLmMyIn0&_nc_ht=scontent-msp1-1.cdninstagram.com&_nc_cat=106&_nc_oc=Q6cZ2QFn-JxBVLuYZE8tEpvhbD1WzxUrYtjHZ0ZGLxaXNqIX8MKW6PR3mjTL_SX1M-9Dqhw&_nc_ohc=uBayL3uahOIQ7kNvwEqJdpB&_nc_gid=kpyqljZx3i9LdXfw47mwkw&edm=AOQ1c0wBAAAA&ccb=7-5&oh=00_AfsKpkb ...


## 4. Evitar duplicados por username

Solo agregamos perfiles cuyo `username` **no** esté ya en `perfilesSeguidores`.

In [5]:
# Usernames ya presentes en perfilesSeguidores
usernames_existentes = {p.get('username', '').lower() for p in perfiles if p.get('username')}

# Filtrar: solo los que NO están
nuevos = []
duplicados = []

for p in faltantes_norm:
    username = p.get('username', '').strip()
    if not username:
        continue
    if username.lower() in usernames_existentes:
        duplicados.append(username)
    else:
        nuevos.append(p)
        usernames_existentes.add(username.lower())

print(f'✅ Perfiles nuevos a agregar: {len(nuevos)}')
print(f'⏭️ Ya existían (se omiten): {len(duplicados)}')
if duplicados:
    print(f'   → {duplicados}')

✅ Perfiles nuevos a agregar: 0
⏭️ Ya existían (se omiten): 16
   → ['mora.dj', 'rropainnteriormasculina', 'remodelaciononline', 'sourkid_aka_mangoroots', 'burdo_tattoo_medellin', 'smoking_buda', 'sebasfili', 'simoonaher', 'samuel_arango49', 'elcantinerofiel', 'clickurbanomedellin', 'mimamayasabesupply', 'riderszz_007', 'jlizrodriguezb', 'gimo.online', 'mauroherrera.a']


## 5. Fusionar y guardar

In [6]:
# Fusionar: perfiles actuales + nuevos
perfiles_final = perfiles + nuevos

# Guardar JSON
with open(PERFILES_JSON, 'w', encoding='utf-8') as f:
    json.dump(perfiles_final, f, ensure_ascii=False, indent=2)

# Guardar CSV
df = pd.DataFrame(perfiles_final)
df.to_csv(PERFILES_CSV, index=False, encoding='utf-8')

print(f'💾 Guardado perfilesSeguidores.json ({len(perfiles_final)} perfiles)')
print(f'💾 Guardado perfilesSeguidores.csv ({len(perfiles_final)} filas)')
print(f'➕ Se agregaron {len(nuevos)} perfiles nuevos')

💾 Guardado perfilesSeguidores.json (8855 perfiles)
💾 Guardado perfilesSeguidores.csv (8855 filas)
➕ Se agregaron 0 perfiles nuevos


## 6. Verificación

In [7]:
# Verificar que los nuevos estén en el archivo
with open(PERFILES_JSON, 'r', encoding='utf-8') as f:
    verif = json.load(f)

usernames_final = {p.get('username') for p in verif}
faltantes_usernames = {p.get('username') for p in faltantes_norm if p.get('username')}

todos_incorporados = faltantes_usernames.issubset(usernames_final)
print('✅ Verificación: todos los 16faltantes están en perfilesSeguidores' if todos_incorporados else '⚠️ Algunos no se incorporaron (posiblemente duplicados)')

# Resumen final
df_verif = pd.DataFrame(verif)
print(f'\n📊 Total de perfiles: {len(verif)}')
df_verif.head(3)

✅ Verificación: todos los 16faltantes están en perfilesSeguidores

📊 Total de perfiles: 8855


,fullName,profilePicUrl,username,postsCount,followersCount,followsCount,private,verified,isBusinessAccount,biography
0,Mauro Sanchez Muñoz,https://scontent-lga3-3.cdninstagram.com/v/t51...,maurosanchezmunoz,5.0,235.0,3455.0,True,False,False,
1,Santiago,https://instagram.fcgy1-2.fna.fbcdn.net/v/t51....,santi.toro26,0.0,1.0,36.0,True,False,False,
2,Jorge Orozco,https://scontent-lga3-3.cdninstagram.com/v/t51...,jao1888n,0.0,146.0,4486.0,False,False,False,


## 7. (Opcional) Actualizar perfiles existentes con datos de 16faltantes

Si los 16 ya estaban en perfilesSeguidores pero querés **sobrescribir** con la versión de 16faltantes (datos más recientes o completos), ejecutá esta celda.

In [8]:
# OPCIONAL: Actualizar perfiles existentes con datos de 16faltantes
# Cargar de nuevo los datos actuales (por si ya se guardó antes)
with open(PERFILES_JSON, 'r', encoding='utf-8') as f:
    perfiles_act = json.load(f)

# Crear índice por username para búsqueda rápida
idx = {p.get('username','').lower(): i for i, p in enumerate(perfiles_act) if p.get('username')}

actualizados = 0
for p in faltantes_norm:
    u = p.get('username','').strip().lower()
    if u and u in idx:
        perfiles_act[idx[u]] = p
        actualizados += 1

# Guardar
with open(PERFILES_JSON, 'w', encoding='utf-8') as f:
    json.dump(perfiles_act, f, ensure_ascii=False, indent=2)
pd.DataFrame(perfiles_act).to_csv(PERFILES_CSV, index=False, encoding='utf-8')

print(f'🔄 Actualizados {actualizados} perfiles con datos de 16faltantes')

🔄 Actualizados 16 perfiles con datos de 16faltantes
